In [3]:
import openai
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import tiktoken
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from operator import itemgetter
from langchain.schema import StrOutputParser
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

import langchain_community.embeddings.huggingface
# help(langchain_community.embeddings.huggingface)
from langchain_community.embeddings.huggingface import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import FAISS

from langchain.chains import LLMChain
from langchain.chains.conversation.memory import ConversationBufferMemory, ConversationBufferWindowMemory, ConversationSummaryMemory, ConversationSummaryBufferMemory

import os, dotenv
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
# embeddings = OpenAIEmbeddings(deployment="textembedding", chunk_size = 16, api_key = os.environ["OPENAI_API_KEY"])
# index_name = "SCLC"

model_name = "bge-large-en-v1.5"
model_kwargs = {"device": "cpu"}
# model_kwargs = {"device": "cuda"}
encode_kwargs = {"normalize_embeddings": True}
embeddings = HuggingFaceBgeEmbeddings(
    # model_name=model_name, 
    model_kwargs = model_kwargs,
    encode_kwargs = encode_kwargs)

index_name = "indexes/ChestPainRubrics"

# store = FAISS.load_local(index_name, embeddings)
import db_firestore as db
store = db.get_store(index_name, embeddings=embeddings)

In [28]:
TEMPLATE = """You are a teacher for medical students. Your task is to provide an overall assessment of a student's diagnosis, based on the 
You will be provided with the following information:
1. The rubrics that the student should be judged based upon.
2. The conversation history between the medical student and the patient.
3. The final diagnosis that the student will make.

Your grading should touch on every part of the rubrics, and grade the student holistically.
Finally, provide an overall grade for the student.

Here is the rubrics:
{context}

----------------------------------------------------------------
You are to give a comprehensive judgement based on the student's diagnosis, with reference to the above rubrics.

Here is the chat history, enclosed in triple backticks:
```
{chat_history}
```
----------------------------------------------------------------

Student's final diagnosis:
{question}
----------------------------------------------------------------
Your grade:
"""

prompt = PromptTemplate(
    input_variables = ["question", "context"],
    template = TEMPLATE
)
retriever = store.as_retriever(search_type="similarity", search_kwargs={"k":2})
def format_docs(docs):
    return "\n--------------------\n".join(doc.page_content for doc in docs)


llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)



## some code to generate the fake history easier

In [29]:
fake_history = []
i = " "

### Send no message to end it
while i:
    i = input()
    if i:
        fake_history.append(i)

print(fake_history)

[]


In [30]:
## Sample history:

fake_history = [
    "student: How are you mr tan?",
    "patient: I am not feeling well...",
    "student: oh, that's terrible! tell me more about what happened to you",
    "patient: well I got this chest pain",
    "student: I'm sorry to hear that. How long has your chest pain been?",
    "patient: I can't remember",
]

In [31]:
## RESET MEMORY
## To reset the memory (if it screws up), rerun this cell
memory = ConversationSummaryBufferMemory(llm=llm, memory_key="chat_history", input_key="question" )

chain = (
    {
        "context": retriever | format_docs, 
        "question": RunnablePassthrough(),
        "chat_history": lambda x: '\n'.join(fake_history)
        } | 
    # prompt | 
    LLMChain(llm=llm, prompt=prompt, memory=memory, verbose=True) #| 
    # StrOutputParser()
)

In [24]:
results = chain.invoke("I believe he has chest pain, but it will go away soon")
print(results.get("text"))

Based on the provided rubrics and the conversation history, I would assess the student's diagnosis as Grade E. 

The student's diagnosis of "chest pain, but it will go away soon" does not align with the symptoms and history provided by the patient. The patient's symptoms, such as acute tearing chest pain, radiation to the back, diaphoresis, syncope, and dysarthria, are indicative of a more serious condition, such as aortic dissection or acute coronary syndrome. The student did not consider these possibilities or explore them further.

Additionally, the student did not adequately explore the patient's presenting complaints, associated symptoms, and relevant medical history. They did not elicit all the necessary information and did not rule out red flags. The student also did not present a comprehensive list of differentials with adequate justification.

Overall, the student's diagnosis and assessment of the patient's condition are incomplete and do not meet the expected standards.
